In [2]:
import pandas as pd
import itertools as it
import gurobipy as gp
from gurobipy import GRB

In [102]:
data1 = [[100, 250, 95, 160],
         [150, 143, 195, 99],
         [135, 80, 242, 55],
         [83, 225, 111, 96],
         [120, 210, 70, 115],
         [230, 98, 124, 80]]

data2 = [[0.6, 0.20, 0.10, 0.10],
         [0.15, 0.55, 0.25, 0.05],
         [0.15, 0.20, 0.54, 0.11],
         [0.08, 0.12, 0.27, 0.53]]

data3 = [[0, 20, 30, 50],
         [20, 0, 15, 35],
         [30, 15, 0, 25],
         [50, 35, 25, 0]]

data4 = [[50, 70],
         [70, 100],
         [120, 150]]


rentDays = [1, 2, 3]
days = [0, 1, 2, 3, 4, 5]
sat = days[-1]
nDay = dict(zip(days, [1, 2, 3, 4, 5, 0]))
pDay = dict(zip(days, [5, 0, 1, 2, 3, 4]))
daysName = dict(zip(days, ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat']))
depots = ['Glasgow', 'Manchester', 'Birmingham', 'Plymouth']

D = pd.DataFrame(data1, columns=depots, index=days)
P = pd.DataFrame(data2, columns=depots, index=depots)
C = pd.DataFrame(data3, columns=depots, index=depots)
Q = dict(zip(rentDays, [0.55, 0.2, 0.25]))

R  = dict(zip(depots, [0, 12, 20, 0]))                              ###########################

RC = pd.DataFrame(data4, columns=['same', 'another'], index=rentDays)
RCSat = dict(zip(['same', 'another'], [30, 50]))
CS = dict(zip(rentDays, [20, 25, 30]))

expanIndex = ['M1', 'M2', 'B1', 'B2', 'P']
expanCap = 5
expanCosts = dict(zip(expanIndex, [18000, 8000, 20000, 5000, 19000]))

In [103]:
model = gp.Model('CarRental1')

# add vars
n  = model.addVar(name='n') # total number of cars

nu = model.addVars(depots, days, name='nu') # undamaged cars at i at day t
nd = model.addVars(depots, days, name='nd') # damaged cars at i at day t

tr = model.addVars(depots, days, name='tr') # rented out cars from i at day t

eu = model.addVars(depots, days, name='eu') # undamaged cars left at i at t
ed = model.addVars(depots, days, name='ed') # damaged cars left at i at t

tu = model.addVars(((i,j,t) for i in depots for j in depots for t in days if i!=j), name='tu') # transfered undamaged cars from i to j at t
td = model.addVars(((i,j,t) for i in depots for j in depots for t in days if i!=j), name='td') # transfered damaged cars from i to j at t

rp = model.addVars(depots, days, name='rp') # to be repaired at i at t

s = model.addVars(expanIndex, vtype=GRB.BINARY, name='s')

for i in depots:
    for t in days:
        tr[i,t].ub = D[i][t]

# add constraints

# repair capacities
for t in days:
    model.addConstr((rp['Glasgow',t] <= R['Glasgow']), name='rep_Gla')
    model.addConstr((rp['Manchester',t] <= R['Manchester'] + expanCap*s['M1'] + expanCap*s['M2']), name='rep_Man')
    model.addConstr((rp['Birmingham',t] <= R['Birmingham'] + expanCap*s['B1'] + expanCap*s['B2']), name='rep_Bir')
    model.addConstr((rp['Plymouth',t] <= R['Plymouth'] + expanCap*s['P']), name='rep_Ply')

    # number of UNDAMAGED cars INTO REPAIR depot i on day t
model.addConstrs((gp.quicksum(0.9*P[i][j]*Q[k]*tr[j,days[t-k]] 
                              for j in depots for k in rentDays) 
                  + gp.quicksum(tu[j,i,days[t-1]] 
                                for j in depots if j != i)
                  + rp[i,days[t-1]]
                  + eu[i,days[t-1]]
                  == nu[i,t] for i in depots for t in days),
                name='R_UN_INTO')

    # number of UNDAMAGED cars OUT REPAIR depot i on day t
model.addConstrs((tr[i,t] 
                  + gp.quicksum(tu[i,j,t] for j in depots if j != i)
                  + eu[i,t]
                  == nu[i,t] for i in depots for t in days),
                name='R_UN_OUT')

    # number of DAMAGED cars INTO REPAIR depot i on day t
model.addConstrs((gp.quicksum(0.1*P[i][j]*Q[k]*tr[j,days[t-k]] 
                              for j in depots for k in rentDays)
                  + gp.quicksum(td[j,i,days[t-1]] 
                                for j in depots if i != j)
                  + ed[i,days[t-1]]
                  == nd[i,t] for i in depots for t in days),
                name='R_DA_INTO')

    # number of DAMAGED cars OUT REPAIR depot i on day t
model.addConstrs((rp[i,t]
                  + gp.quicksum(td[i,j,t] 
                                for j in depots if i != j)
                  + ed[i,t]
                  == nd[i,t] for i in depots for t in days),
                name='R_DA_OUT')

# limiting the total number of cars
model.addConstr((gp.quicksum(0.25*tr[i,0] + 0.45*tr[i,1]
                             + nu[i,2] + nd[i,2] for i in depots)
                == n), name='totalCars')

# max of three expansions can be chosen
model.addConstr((gp.quicksum(s[i] for i in expanIndex) <=3),
               name='3_expan')

# if second expansion for a given depot is chosen, then first expansion should be chosen too
model.addConstr((s['M1'] >= s['M2']),
               name='Man_expan')
model.addConstr((s['B1'] >= s['B2']),
               name='Bir_expan')

# objective function
model.setObjective((gp.quicksum(P[i][i]*Q[k]*(RC['same'][k] - CS[k] + 10)*tr[i,t] 
                                for i in depots for t in days[:6] for k in rentDays)
                   + gp.quicksum(P[j][i]*Q[k]*(RC['another'][k] - CS[k] + 10)*tr[i,t] 
                                 for i in depots for j in depots for t in days[:6] for k in rentDays if i!=j)
                   #+ gp.quicksum(P[i][i]*Q[1]*(RCSat['same'] - CS[1] + 10)*tr[i,sat] 
                   #              for i in depots)
                   #+ gp.quicksum(P[j][i]*Q[1]*(RCSat['another'] - CS[1] + 10)*tr[i,sat] 
                   #              for i in depots for j in depots if j!=i) 
                   #+ gp.quicksum(P[i][i]*Q[k]*(RC['same'][k] - CS[k] + 10)*tr[i,sat] 
                   #              for i in depots for k in rentDays[1:])
                   #+ gp.quicksum(P[j][i]*Q[k]*(RC['another'][k] - CS[k] + 10)*tr[i,sat] 
                   #              for i in depots for j in depots for k in rentDays[1:] if j!=i)
                   - gp.quicksum(C[j][i]*tu[i,j,t] 
                                 for i in depots for j in depots for t in days if j!=i)
                   - gp.quicksum(C[j][i]*td[i,j,t] 
                                 for i in depots for j in depots for t in days if j!=i)
                   - 15*n
                   - gp.quicksum(expanCosts[i]*s[i] for i in expanIndex)),
                  GRB.MAXIMIZE)

model.update()
model.write('Car Rental II.lp')
model.Params.DualReductions = 1
model.optimize()

Parameter DualReductions unchanged
   Value: 1  Min: 0  Max: 1  Default: 1
Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 124 rows, 294 columns and 1208 nonzeros
Model fingerprint: 0xdeea7297
Variable types: 289 continuous, 5 integer (5 binary)
Coefficient statistics:
  Matrix range     [1e-03, 5e+00]
  Objective range  [2e+01, 2e+04]
  Bounds range     [1e+00, 3e+02]
  RHS range        [3e+00, 2e+01]
Found heuristic solution: objective -0.0000000
Presolve removed 55 rows and 55 columns
Presolve time: 0.00s
Presolved: 69 rows, 239 columns, 1077 nonzeros
Variable types: 234 continuous, 5 integer (5 binary)

Root relaxation: objective 1.368577e+05, 85 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 136857.699    0    2   -0.00000 136857.699    

In [104]:
print('Profit of ${:,.2f}'.format(model.ObjVal))
print('Number of owned cars is {:.0f}'.format(n.x))

report_rented = pd.DataFrame([], columns=depots, index=list(daysName.values()))

for i,t in tr.keys():
    report_rented[i][daysName[t]] = round(tr[i,t].x)
report_rented

Profit of $132,136.73
Number of owned cars is 890


,Glasgow,Manchester,Birmingham,Plymouth
Mon,86,168,95,51
Tue,86,135,195,50
Wed,90,80,242,54
Thu,83,192,111,57
Fri,102,150,70,56
Sat,91,98,124,54


In [105]:
report_damag = pd.DataFrame([], columns=depots, index=list(daysName.values()))
for i,t in nd.keys():
    report_damag[i][daysName[t]] = round(nd[i,t].x)
report_damag

,Glasgow,Manchester,Birmingham,Plymouth
Mon,10,22,20,9
Tue,10,22,20,6
Wed,10,22,20,6
Thu,11,22,20,8
Fri,13,22,23,6
Sat,14,22,22,6


In [106]:
report_undamag = pd.DataFrame([], columns=depots, index=list(daysName.values()))
for i,t in nu.keys():
    report_undamag[i][daysName[t]] = round(nu[i,t].x)
report_undamag

,Glasgow,Manchester,Birmingham,Plymouth
Mon,86,168,265,51
Tue,86,135,291,50
Wed,90,137,242,54
Thu,93,192,161,57
Fri,102,150,201,56
Sat,91,137,264,54
